In [ ]:
from openai import OpenAI
import gradio
import re
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display


In [ ]:
os.environ['PATH']

In [8]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [9]:
client = OpenAI()

In [10]:
system_cpp_prompt = """
You are an expert in converting Python code to C++. You are highly skilled in understanding Python syntax and logic, and you are capable of
translating that into efficient, well-structured C++ code while ensuring it maintains the same functionality and behavior.
Please carefully analyze the provided Python code and provide an accurate and optimized C++ translation."
"""

In [59]:
system_python_prompt = """
You are an expert in extracting Python code from text. Your task is to identify and extract all Python code snippets within the given text.
Please carefully locate any Python code, which may include functions, classes, variables, and other syntax, and extract it accurately.
Do not add any additional text or explanation or any greeting, only provide the extracted Python code in its original format.
"""


In [60]:
def model_generation(system_prompt, user_prompt):

  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[{"role":"system","content": system_prompt}
          ,{"role":"user", "content":user_prompt}]
  )
  return completion.choices[0].message.content

In [ ]:

def extracting_code_from_model_response(langage, response):
    try:
        match = re.search(rf"```{langage}(.*?)```", response, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            raise ValueError(f"No code {langage} found in the response.")
    except Exception as e:
        return f"Error: {str(e)}"

In [136]:
def processus_extraction(code):

  python_response = model_generation(system_python_prompt,code)
  python_code = extracting_code_from_model_response("python", python_response)

  cpp_response = model_generation(system_cpp_prompt,python_code)
  cpp_code = extracting_code_from_model_response('cpp', cpp_response)
  markdown_cpp =  f"```\n{cpp_code}\n```"
  return markdown_cpp

In [ ]:


backgroud_color = ".gradio-container {background-color: #f0f0f0;}"
with gr.Blocks(css=backgroud_color) as demo:

  with gr.Row():
    with gr.Column():
      gr.Markdown("## Enter the python code here")
      code = gr.Textbox(label="Python code")
      greet_button = gr.Button("Greet")


    with gr.Column():
      gr.Markdown("## C++ Code")
      output = [gr.Markdown()]


  greet_button.click(fn=processus_extraction, inputs=[code], outputs=output)


demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e6d4c973f145d8df04.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
